In [1]:
import numpy as np
import pandas as pd

In [2]:
import MySQLdb

In [3]:
import matplotlib.pyplot as plt
import seaborn as sns

In [4]:
import warnings
warnings.simplefilter("ignore")

### Input Data

In [5]:
def get_data(filename):
    #change for sharing
    with open("D:/github_projects/login.txt") as file:
        myserver_config = file.read().split(",") #config data for SQL server
        connect = MySQLdb.connect(*myserver_config)
        cursor = connect.cursor()
        query = f"SELECT * FROM {filename}"
        data = pd.read_sql(query, connect, index_col="id")

In [6]:
def change_types(mydata):
    mydata = mydata.replace("None", np.nan)
    for elem in ["Density", "Х", "У", "Scanning_spot"]:
        mydata[elem] = mydata[elem].str.replace(",", ".")
        mydata[elem] = mydata[elem].astype(np.float32)
    mydata["Lazer"] = mydata["Lazer"].astype(np.object_)
    #remove thickness due to its various
    mydata.drop("Thickness", inplace=True, axis=1)
    return mydata


In [7]:
data = change_types(data)

### Data Preprocessing

Main data description

In [18]:
class DataPreprocessing:
    def __init__(self, df):
        self.__data = df
        self.numerical_cols = df.select_dtypes(["int", "float"]).columns
        self.categorical_cols = df.select_dtypes(["object"]).columns
        self.__X = df.drop("Density", axis=1)
        self.__y = df["Density"]
    
    def Main_info(self):
        print(self.head())
        print(self.info())
        print(self.describe())    

    def Numerical_review(self):
        for col in self.numerical_cols:
            sns.histplot(self.__data[col], kde=True)
            plt.title(f"Distribution of {col}")
            plt.show();

    def Emmisions_review(self):
        for col in self.numerical_cols:
            sns.boxplot(self.__data[col])
            plt.title(f"Boxplot of {col}")
            plt.show();

    def Correlation_review(self):
        corr_matrix = self.__data.select_dtypes(["int", "float"]).corr()
        sns.heatmap(data=corr_matrix, annot=True, cmap="coolwarm");

    def Categorical_review(self):
        for col in self.categorical_cols:
            sns.countplot(x = col, data=df)
            plt.title(f"Countplot of {col}")
            plt.show();

    def One_hot_encoding(self):
        return 

SyntaxError: incomplete input (719412577.py, line 36)

In [13]:
def One_hot_encoding(df):
    return pd.get_dummies(data=df, columns=categorical_columns, drop_first=True)

Standartization

In [14]:
from sklearn.preprocessing import StandardScaler

In [15]:
# Split in target and values
X = data.drop("Density", axis=1)
y = data["Density"]

In [16]:
numerical_columns = list(numerical_columns)
numerical_columns.remove("Density")

NameError: name 'numerical_columns' is not defined

In [ ]:
scaler = StandardScaler()
X[numerical_columns] = scaler.fit_transform(X[numerical_columns])